## Compare Synthetic with Observations

per pixel error estimates


In [ ]:
import folium
import xskillscore as xs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb

import odc.geo.xr
from odc.geo.xr import assign_crs

In [ ]:
model_var='LST'

In [ ]:
yy = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/results/ml_predictions/'+model_var+'_predicted_5km_monthly_1982_2022.nc')
obs = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/'+model_var+'_5km_monthly_1982_2022_wGaps.nc')

if model_var=='LST':
    yy = yy-273.15
    obs = obs-273.15

obs = obs.sel(time=yy.time)
obs = obs[model_var].rename({'latitude':'y', 'longitude':'x'})
gaps_mask = ~np.isnan(obs)

In [ ]:
diff = yy[model_var] - obs

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(13,5))
yy[model_var].where(gaps_mask).mean(['x','y']).plot(ax=ax, label='predictions')
obs.mean(['x','y']).plot(ax=ax, label='observed')
# ax.set_ylim(0.15, 0.40)
ax.legend()
ax.set_title('Aus-Wide '+model_var+ ' matched data gaps');

### Error metrics

In [ ]:
corr = xr.corr(obs, yy[model_var], dim='time')
mape = xs.mape(yy[model_var], obs, dim='time', skipna=True)
mape = mape*100
mae = xs.mae(yy[model_var], obs, dim='time', skipna=True)

mape = mape.rename('MAPE (%)')
mae = mae.rename('MAE')
corr = corr.rename('Pearson Correlation')

In [ ]:
pred_mean = yy[model_var].mean('time')
obs_mean = obs.mean('time')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5), sharey=True)
mae.plot.imshow(ax=ax[0], robust=True, add_labels=False, cmap='cividis')
mape.plot.imshow(ax=ax[1], robust=True, add_labels=False, cmap='inferno')
corr.plot.imshow(ax=ax[2], cmap='magma', vmin=0.5, vmax=1.0, add_labels=False)
ax[0].set_title('Mean Absolute Error')
ax[1].set_title('Mean Absolute Percentage Error')
ax[2].set_title('Pearson Correlation Coefficient')
plt.tight_layout()

fig.savefig("/g/data/os22/chad_tmp/climate-carbon-interactions/results/figs/Error_metrics_"+model_var+"_lgbm.png",
            bbox_inches='tight', dpi=300)

In [ ]:
m = folium.Map(control_scale = True)

# assign_crs(corr, crs='EPSG:4326').odc.add_to(m, vmin=0.5, vmax=1.0, cmap='magma')
assign_crs(mae, crs='EPSG:4326').odc.add_to(m, robust=True, cmap='inferno')

# Zoom map to Australia
m.fit_bounds(corr.odc.map_bounds())

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
       ).add_to(m)

folium.LayerControl().add_to(m)
display(m)